## Example of Computing Implicit Prices from Consumption & Expenditures



In [1]:
import cfe
import pandas as pd
import numpy as np

Sheets = {"ICRISAT_wprices":"13Ig5hZif-NSHtgkKRp_cEgKXk0lOsdUB2BAD6O_FnRo"}

def dataframe_from_gspreadsheet(sheet_name, key):
    """Transform public google spreadsheet into pandas.DataFrame."""
    
    url = 'https://docs.google.com/spreadsheets/d/{key}/gviz/tq?tqx=out:csv&sheet={sheet_name}&headers=1'.format(
        key=key, sheet_name=sheet_name.replace(' ', '%20'))

    print(url)
    df = pd.read_csv(url,dtype={'j':str})

    df.columns = [c.strip() for c in df.columns.tolist()]

    df = df.loc[:,~df.columns.duplicated(keep='first')]   

    df = df.drop([col for col in df.columns if col.startswith('Unnamed')], axis=1)

    df = df.loc[~df.index.duplicated(), :]

    return df

def prices(Q,X,tol=1e-6):
    """Impute prices from data on expenditures and quantities.

    Non-trivial because quantities may be reported in different units.
    """
    
    myQ = Q.groupby(['j','t','m','unit']).sum()

    B={}
    for t in myQ.index.levels[1]:
        for m in myQ.index.levels[2]:
            for i in myQ.columns:
                try:
                    useX = X.query("t==%d and m=='%s'" % (t,m))[i].fillna(0)
                    useQ = myQ.query("t==%d and m=='%s'" % (t,m))[i].fillna(0).unstack('unit')
                    if len(useX):
                        q,x = useQ.fillna(0).align(useX.fillna(0),axis=0,join='inner')
                        b = np.linalg.lstsq(q,x,rcond=None)[0]
                        b = pd.Series(b,index=q.columns,name=i)
                        B[(t,m,i)] = b.where(b>0,0).round(6)
                except KeyError:  # i not in X?
                    pass

    P = pd.concat(B).replace(0,np.nan).dropna()
    P.index.names = ['t','m','i','unit']

    P = P.unstack(['i','unit'])
    
    return P

for k,v in Sheets.items():
    print(k)
    x = dataframe_from_gspreadsheet("Expenditures", Sheets[k])
    c = dataframe_from_gspreadsheet("Consumption", Sheets[k])

    z = dataframe_from_gspreadsheet("Household Characteristics", Sheets[k])

    # If no 'm' index assume a single market:
    if 'm' not in z.index.names and 'm' not in z.columns:
        z['m'] = 1
        x['m'] = 1
        c['m'] = 1

    z = z.reset_index().set_index(['j','t','m'])
    c = c.reset_index().set_index(['j','t','m'])
    x = x.reset_index().set_index(['j','t','m'])

    x = x.loc[~x.index.duplicated(), :]
    c = c.loc[~c.index.duplicated(), :]
    z = z.loc[~z.index.duplicated(), :]

    p = prices(c,x)  # Prices for all transactions

    # Median price for different goods in different units
    # in different markets and in different years 
    p.groupby(['m','t']).median()